In [1]:
# Use the Azure Machine Learning data collector to log various metrics
from azureml.logging import get_azureml_logger
logger = get_azureml_logger()

In [2]:
# Use Azure Machine Learning history magic to control history collection
# History is off by default, options are "on", "off", or "show"
# %azureml history on

In [3]:
import sys, os, os.path, glob
import tensorflow as tf
from keras import backend as K
from keras.applications import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.models import Model

Using TensorFlow backend.


In [4]:
def setup_dnn():
    model = InceptionV3(weights="imagenet", include_top=False) #include_top=False excludes final FC layer
    return model

def setup_transfer_learninig(model, base_model):
    """Freeze all layers and compile the model"""
    for layer in base_model.layers:
        layer.trainable = False  
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

def setup_new_classifier(base_model, nb_classes):
    """Add last layer to the convnet
        Args:
            base_model: keras model excluding top
            nb_classes: # of classes
        Returns:
            new keras model with last layer
    """
    FC_SIZE = 1024
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(FC_SIZE, activation='relu')(x) #new FC layer, random init
    predictions = Dense(nb_classes, activation='softmax')(x) #new softmax layer
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

def setup_finetune(model):
    """Freeze the bottom NB_IV3_LAYERS and retrain the remaining top layers.
        note: NB_IV3_LAYERS corresponds to the top 2 inception blocks in the inceptionv3 arch
        Args:
            model: keras model
    """
    NB_IV3_LAYERS_TO_FREEZE = 172
    
    for layer in model.layers[:NB_IV3_LAYERS_TO_FREEZE]:
        layer.trainable = False
    for layer in model.layers[NB_IV3_LAYERS_TO_FREEZE:]:
        layer.trainable = True
    model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
    
def save_dnn(model, folder, filename):
    filepath = os.path.join(folder, filename)
    sess = K.get_session()
    tf.train.write_graph(sess.graph.as_graph_def(), folder, filename, as_text=False)
    tf.train.export_meta_graph()
    print('saved the graph definition in tensorflow format at: ', filepath)

def utils_files_count(directory):
    """Get number of files by searching directory recursively"""
    if not os.path.exists(directory):
        return 0
    cnt = 0
    for r, dirs, files in os.walk(directory):
        for dr in dirs:
            cnt += len(glob.glob(os.path.join(r, dr + "/*")))
    return cnt

def train_generator(folder, batch_size=16, save_to_dir=None):
    IM_WIDTH, IM_HEIGHT = 299, 299 #fixed size for InceptionV3
    datagen =  ImageDataGenerator(
        preprocessing_function=preprocess_input,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )
    #generated_folder = os.path.join(folder, '..', 'generated')
    if save_to_dir:
        if not os.path.exists(save_to_dir):
            os.makedirs(save_to_dir)
        else:
            utils_removeFilesInFolder(save_to_dir)

    generator = datagen.flow_from_directory(
        folder,
        target_size=(IM_WIDTH, IM_HEIGHT),
        batch_size=batch_size,
        save_to_dir=save_to_dir
    )
    return generator

def validation_generator(folder, batch_size=16):
    IM_WIDTH, IM_HEIGHT = 299, 299 #fixed size for InceptionV3
    datagen =  ImageDataGenerator(
        preprocessing_function=preprocess_input,
        rotation_range=45,
        width_shift_range=0,
        height_shift_range=0,
        shear_range=0,
        zoom_range=0,
        horizontal_flip=True
    )
    
    generator = datagen.flow_from_directory(
        folder,
        target_size=(IM_WIDTH, IM_HEIGHT),
        batch_size=batch_size
    )
    return generator

def utils_removeFilesInFolder(folder):
    files = glob.glob(os.path.join(folder,'*'))
    for f in files:
        os.remove(f)

def utils_generated_sample (data_folder, generated_folder, batches_length=1):
    generator = train_generator(data_folder, batch_size=16, save_to_dir=generated_folder)
    
    i = 0
    for batch in generator:
        i += 1
        if i > batches_length:
            break  # otherwise the generator would loop indefinitely
    

In [5]:
project_folder = %pwd
print("Project folder: ", project_folder)
output_folder = os.path.join(project_folder,'output')
data_folder = os.path.join(project_folder, 'data')
train_folder = os.path.join(data_folder, 'train')
validation_folder = os.path.join(data_folder, 'validation')
nb_classes = len(glob.glob(train_folder + "/*"))
batch_size=16
nb_epoch = 32
# load weights
model_filename = 'model.pb'
#if not os.path.exists(output_folder):
#    os.makedirs(output_folder)

Project folder:  D:\fran\Work\eShoConAI\workbench


In [6]:
# setup model
base_model = setup_dnn()
model      = setup_new_classifier(base_model, nb_classes)

# transfer learning
setup_transfer_learninig(model, base_model)

history_tl = model.fit_generator(
    train_generator(train_folder,batch_size),
    steps_per_epoch=500//batch_size, #utils_files_count(train_folder)//batch_size,
    epochs=32,
    validation_data=validation_generator(validation_folder, batch_size),
    validation_steps=100//batch_size, #utils_files_count(validation_folder)//batch_size,
    verbose=1,
    class_weight='auto')

Found 72 images belonging to 4 classes.
Found 15 images belonging to 4 classes.
Epoch 1/32
31/31 [==============================] - 22s 695ms/step - loss: 4.3568 - acc: 0.5787 - val_loss: 2.1624 - val_acc: 0.8667
Epoch 2/32
31/31 [==============================] - 16s 532ms/step - loss: 0.9435 - acc: 0.8530 - val_loss: 0.5018 - val_acc: 0.8667
Epoch 3/32
31/31 [==============================] - 16s 529ms/step - loss: 0.2461 - acc: 0.9175 - val_loss: 0.6644 - val_acc: 0.7667
Epoch 4/32
31/31 [==============================] - 16s 522ms/step - loss: 0.4083 - acc: 0.9176 - val_loss: 0.3239 - val_acc: 0.8778
Epoch 5/32
31/31 [==============================] - 16s 524ms/step - loss: 0.5309 - acc: 0.8853 - val_loss: 0.3271 - val_acc: 0.8889
Epoch 6/32
31/31 [==============================] - 16s 529ms/step - loss: 0.3172 - acc: 0.9576 - val_loss: 0.4153 - val_acc: 0.8667
Epoch 7/32
31/31 [==============================] - 16s 505ms/step - loss: 0.1954 - acc: 0.9295 - val_loss: 0.3072 - val_a

In [7]:
save_dnn(model, output_folder, model_filename)

saved the graph definition in tensorflow format at:  D:\fran\Work\eShoConAI\workbench\output\model.pb


In [ ]:
#utils_generated_sample(train_folder, os.path.join(data_folder, 'generated'),2)

In [ ]:
#model.get_weights()

In [8]:
print(history_tl.history.keys())

dict_keys(['val_loss', 'loss', 'acc', 'val_acc'])


In [11]:
import numpy
print ("Accuracy: ", numpy.mean(history_tl.history['acc']))

Accuracy:  0.952715684809


In [12]:
labels = ["bracelet","earrings","parasol","thermometer"]
def load_image(img_path):
    from keras.preprocessing import image
    IM_WIDTH, IM_HEIGHT = 299, 299 #fixed size for InceptionV3
    img = image.load_img(img_path, target_size=(IM_WIDTH, IM_HEIGHT))
    x = image.img_to_array(img)
    x = numpy.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

In [13]:
bracelet_path = os.path.join(validation_folder,'bracelet','3506.jpg')
bracelet_image = load_image(bracelet_path)
preds = model.predict(bracelet_image)
print (preds)
print (labels[preds.argmax()])

[[  9.99994874e-01   5.10283098e-06   1.64173886e-09   1.39166834e-09]]
bracelet


In [14]:
earrings_path = os.path.join(validation_folder,'earrings','3451b.jpg')
earrings_image = load_image(earrings_path)
preds = model.predict(earrings_image)
print (preds)
print (labels[preds.argmax()])

[[  8.95389616e-01   3.06148008e-02   1.69649851e-04   7.38259330e-02]]
bracelet


In [15]:
parasol_path = os.path.join(validation_folder,'parasol','114a.jpg')
parasol_image = load_image(parasol_path)
preds = model.predict(parasol_image)
print (preds)
print (labels[preds.argmax()])

[[  1.09924203e-09   2.18095261e-16   1.00000000e+00   3.17252669e-10]]
parasol


In [16]:
thermo_path = os.path.join(validation_folder,'thermometer','1866a.jpg')
thermo_image = load_image(thermo_path)
preds = model.predict(thermo_image)
print (preds)
print (labels[preds.argmax()])

[[  1.25869898e-17   9.73673029e-23   1.04478805e-18   1.00000000e+00]]
thermometer


In [ ]:
def save_dnn2(model, folder, filename):
    filepath = os.path.join(folder, filename)
    sess = K.get_session()
    graph1 = sess.graph.as_graph_def()
    vars = {}
    for v in tf.trainable_variables():
        vars[v.value().name] = sess.run(v)
    
    with tf.Graph().as_default() as graph2:
        consts = {}
        for k in vars.keys():
            consts[k] = tf.constant(vars[k])
        tf.import_graph_def(graph1,input_map={name:consts[name] for name in consts.keys()})
        
        tf.train.write_graph(sess.graph_def,folder,filename,False)
        #tf.train.write_graph(sess.graph.as_graph_def(), folder, filename, as_text=False)
        print('saved the graph definition in tensorflow format at: ', filepath)

In [ ]:
#save_dnn2(model, output_folder, 'model2.pb')

In [17]:
print (model.input.name)
print (model.output.name)

input_1:0
dense_2/Softmax:0
